In [ ]:
#SARIMAXコード
for station_id in stationid_set:
    #指定した一つの地点のdataframeにしている
    train = main_df[main_df['station_id'] == station_id]
    #学習期間を指定
    train = train[train['date'] < '2013-10-01']
    train = train[train['date'] >= '2013-09-01']
    #学習に不必要な説明変数の削除
    train = train.drop("station_id",axis=1)
    train = train.set_index('date')
    #目的変数以外を外部変数に設定
    train_exog =  train.drop("bikes_available",axis=1)
    
    #予測期間を指定した一つの地点のdataframeにしている
    test = main_df[main_df['station_id'] == station_id]
    #予測期間を指定
    test = test[test['date'] <= '2013-10-08 00:00:00']
    test = test[test['date'] >= '2013-10-01']
    test = test.set_index('date')
    #予測に不必要な外部変数の削除
    test_exog = test.drop("station_id",axis=1)
    test_exog =  test_exog.drop("bikes_available",axis=1)
    #トレンドに関するAR項の数:p
    #トレンドに関する差分の回数:d
    #トレンドに関するMA項の数:q
    #季節変動に関するAR項の数:sp
    #季節変動に関する差分の回数:sd
    #季節変動に関するMA項の数:sq
    p, d, q, sp, sd, sq = 24, 1, 24, 1, 1, 1
    #季節変動の期間範囲を指定する項目数
    seasonal = 48

    # SARIMAXモデルで学習
    result_sarima = SARIMAX(
        endog = train["bikes_available"], 
        exog = train_exog,
        order=(p,d,q), 
        seasonal_order=(sp,sd,sq,seasonal), 
        enforce_stationarity = False, 
        enforce_invertibility = False
    ).fit()
    #学習したモデルで予測期間で予測する
    predict_sarima = result_sarima.predict('2013-10-01', '2013-10-08', exog=test_exog, dynamic=True)

In [ ]:
#RandomForestコード(lag変数あり)
for station_id in stationid_set:
    #指定した一つの地点のdataframeにしている
    train_X = main_df[main_df['station_id'] == station_id]
    #lag変数用のdetaframeの準備
    train_lag = train_X[train_X['date'] < '2013-10-01']['bikes_available']
    #学習期間を指定
    train_learn = train_X[train_X['date'] < '2013-10-01']
    train_learn = train_learn[train_learn['date'] >= '2013-09-02']
    #学習期間の説明変数にlag変数を追加する
    i = 1
    while i < 25:
        row_name = "lag_" + str(i) + "hour"
        set = train_lag[24-i:720-i]
        train_learn[row_name] = set.values
        i += 1
    i = 0    
    #学習期間の目的変数を指定
    train_learn_y = train_learn["bikes_available"]
    #学習期間で不必要な説明変数の削除
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    
    
    #RandomForestで学習
    regr = RandomForestRegressor(max_depth=15, random_state=0)
    regr.fit(train_learn_x, train_learn_y)
    
    #予測期間のlag変数用のdetaframeの準備
    test_lag = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-01']
    test_lag = test_lag[test_lag['date'] >= '2013-09-30']['bikes_available']   
    #予測期間を指定
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-08']
    test = test[test['date'] >= '2013-10-01']
    #予測期間の目的変数を設定
    test_y = test["bikes_available"]
    #予測期間で不必要な説明変数の削除
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    
    
    #予測期間で1時点ずつ予測する
    pred_lightgbm_test = []
    i=0
    while i < 168:
        test_x_line = test_x[i:i+1]
        lag = []
        for j in pred_lightgbm_test:
            lag.insert(0, j)
        for k in range(24 - len(pred_lightgbm_test)):
            lag.append(test_lag[23-k:24-k].values)
        s = 1
        while s < 25:
            row_name = "lag_" + str(s) + "hour"
            test_x_line[row_name] = lag[s-1]
            s += 1
        pred_lightgbm_test_lag_line = regr.predict(test_x_line)
        pred_lightgbm_test =np.append(pred_lightgbm_test, pred_lightgbm_test_lag_line)
        i += 1

In [ ]:
#RandomForestコード(回帰分析)
for station_id in stationid_set:
    #指定した一つの地点のdataframeにしている
    train_X = main_df[main_df['station_id'] == station_id]
    #学習期間を指定
    train_learn = train_X[train_X['date'] < '2013-10-01']
    train_learn = train_learn[train_learn['date'] >= '2013-09-02']
    #学習期間の目的変数を指定
    train_learn_y = train_learn["bikes_available"]
    #学習期間で不必要な説明変数の削除
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    
    
    #RandomForestで学習
    regr = RandomForestRegressor(max_depth=15, random_state=0)
    regr.fit(train_learn_x, train_learn_y)
    
    #予測期間を指定
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-08']
    test = test[test['date'] >= '2013-10-01']
    #予測期間の目的変数を設定
    test_y = test["bikes_available"]
    #予測期間で不必要な説明変数の削除
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)  
    #学習したモデルで予測
    pred_lightgbm_test = regr.predict(test_x)

In [ ]:
#LightGBMコード(lag変数あり)
for station_id in stationid_set:
    #指定した一つの地点のdataframeにしている
    train_X = main_df[main_df['station_id'] == station_id]
    #lag変数用のdetaframeの準備
    train_lag = train_X[train_X['date'] < '2013-10-01']['bikes_available']
    #学習期間を指定
    train_learn = train_X[train_X['date'] < '2013-10-01']
    train_learn = train_learn[train_learn['date'] >= '2013-09-02']
    #学習期間の説明変数にlag変数を追加する
    i = 1
    while i < 25:
        row_name = "lag_" + str(i) + "hour"
        set = train_lag[24-i:720-i]
        train_learn[row_name] = set.values
        i += 1
    i = 0    
    #学習期間の目的変数を指定
    train_learn_y = train_learn["bikes_available"]
    #学習期間で不必要な説明変数の削除
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    

    train_lightgbm_set = lgb.Dataset(train_learn_x, train_learn_y)
    
    #LightGBMで学習
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "verbose" : -1
    }
    lgb_model_24 = lgb.train(
        params = params,
        train_set = train_lightgbm_set,
        valid_sets = [train_lightgbm_set, train_lightgbm_set],
        num_boost_round = 300,
        early_stopping_rounds=10,
        verbose_eval=5
    )
    
    #予測期間のlag変数用のdetaframeの準備
    test_lag = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-01']
    test_lag = test_lag[test_lag['date'] >= '2013-09-30']['bikes_available']     
    #予測期間を指定
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-08']
    test = test[test['date'] >= '2013-10-01']
    #予測期間の目的変数を設定
    test_y = test["bikes_available"]
    #予測期間で不必要な説明変数の削除
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    
    
    #予測期間で1時点ずつ予測する
    pred_lightgbm_test = []
    i=0
    while i < 168:
        test_x_line = test_x[i:i+1]
        lag = []
        for j in pred_lightgbm_test:
            lag.insert(0, j)
        for k in range(24 - len(pred_lightgbm_test)):
            lag.append(test_lag[23-k:24-k].values)
        s = 1
        while s < 25:
            row_name = "lag_" + str(s) + "hour"
            test_x_line[row_name] = lag[s-1]
            s += 1
        pred_lightgbm_test_lag_line = lgb_model_24.predict(test_x_line)
        pred_lightgbm_test =np.append(pred_lightgbm_test, pred_lightgbm_test_lag_line)
        i += 1

In [ ]:
#LightGBMコード(回帰分析)
for station_id in stationid_set:
    #指定した一つの地点のdataframeにしている
    train_X = main_df[main_df['station_id'] == station_id]
    #学習期間を指定
    train_learn = train_X[train_X['date'] < '2013-10-01']
    train_learn = train_learn[train_learn['date'] >= '2013-09-02']
    #学習期間の目的変数を指定
    train_learn_y = train_learn["bikes_available"]
    #学習期間で不必要な説明変数の削除
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    

    train_lightgbm_set = lgb.Dataset(train_learn_x, train_learn_y)
    #LightGBMで学習
    model = lgb.LGBMRegressor(objective='rmse', n_estimators = 300)
    model.fit(train_learn_x, train_learn_y, eval_set=[(train_learn_x, train_learn_y), (train_learn_x, train_learn_y)], verbose=10)
    
    #予測期間を指定
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-08']
    test = test[test['date'] >= '2013-10-01']
    #予測期間の目的変数を設定
    test_y = test["bikes_available"]
    #予測期間で不必要な説明変数の削除
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    
    #学習したモデルで予測
    pred_lightgbm_test = model.predict(test_x) 

In [ ]:
#XGBoostコード(lag変数あり)
for station_id in stationid_set:
    #指定した一つの地点のdataframeにしている
    train_X = main_df[main_df['station_id'] == station_id]
    #lag変数用のdetaframeの準備
    train_lag = train_X[train_X['date'] < '2013-10-01']['bikes_available']
    #学習期間を指定
    train_learn = train_X[train_X['date'] < '2013-10-01']
    train_learn = train_learn[train_learn['date'] >= '2013-09-02']
    #学習期間の説明変数にlag変数を追加する
    i = 1
    while i < 25:
        row_name = "lag_" + str(i) + "hour"
        set = train_lag[24-i:720-i]
        train_learn[row_name] = set.values
        i += 1
    i = 0    
    #学習期間の目的変数を指定
    train_learn_y = train_learn["bikes_available"]
    #学習期間で不必要な説明変数の削除
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    
    
    #XGBoostで学習
    reg_mod = xgb.XGBRegressor(objective='reg:squarederror',
        n_estimators=300,
        learning_rate=0.10,
        subsample=0.5,
        colsample_bytree=1, 
        max_depth=15,
    )
    reg_mod.fit(train_learn_x, train_learn_y) 

    #予測期間のlag変数用のdetaframeの準備
    test_lag = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-01']
    test_lag = test_lag[test_lag['date'] >= '2013-09-30']['bikes_available'] 
    #予測期間を指定
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-08']
    test = test[test['date'] >= '2013-10-01']
    #予測期間の目的変数を設定
    test_y = test["bikes_available"]
    #予測期間で不必要な説明変数の削除
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    
    
    #予測期間で1時点ずつ予測する
    pred_lightgbm_test = []
    i=0
    while i < 168:
        test_x_line = test_x[i:i+1]

        lag = []
        for j in pred_lightgbm_test:
            lag.insert(0, j)

        for k in range(24 - len(pred_lightgbm_test)):
            lag.append(test_lag[23-k:24-k].values)
        s = 1
        while s < 25:
            row_name = "lag_" + str(s) + "hour"
            test_x_line[row_name] = lag[s-1]
            s += 1

        pred_lightgbm_test_lag_line = reg_mod.predict(test_x_line)
        pred_lightgbm_test =np.append(pred_lightgbm_test, pred_lightgbm_test_lag_line)
        i += 1

In [ ]:
#XGboostコード(回帰分析)
for station_id in stationid_set:
    #指定した一つの地点のdataframeにしている
    train_X = main_df[main_df['station_id'] == station_id]
    #学習期間を指定
    train_learn = train_X[train_X['date'] < '2014-02-01']
    train_learn = train_learn[train_learn['date'] >= '2014-01-02']
    #学習期間の目的変数を指定
    train_learn_y = train_learn["bikes_available"]
    #学習期間で不必要な説明変数の削除
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    
    
    #XGBoostで学習
    dtrain = xgb.DMatrix(train_learn_x, label=train_learn_y)
    params = {
        "objective" : "reg:squarederror",
        "eval_metric" : "rmse"
    }        
    dtrain = xgb.DMatrix(train_learn_x, train_learn_y)
    results_dict = {}
    model = xgb.train(
        params = params,
        dtrain =  dtrain,
        evals = [(dtrain, "train"), (dtrain, "train")],
        num_boost_round = 300,
        verbose_eval=50,
        evals_result = results_dict
    )
    #予測期間の指定
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2014-02-08']
    test = test[test['date'] >= '2014-02-01']
    #予測期間の目的変数を設定
    test_y = test["bikes_available"]
    #予測期間で不必要な説明変数の削除
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    
    #学習したモデルで予測
    pred_lightgbm_test = model.predict(xgb.DMatrix(test_x))

In [ ]:
#LSTMコード
for station_id in stationid_set:
    #指定した一つの地点のdataframeにしている
    train = main_df[main_df['station_id'] == station_id]
    #学習期間を指定
    train = train[train['date'] < '2013-10-01']
    train = train[train['date'] >= '2013-09-01']
    #学習期間で不必要な説明変数の削除
    train = train.drop("station_id",axis=1)
    
    #指定した一つの地点のdataframeにしている
    test = main_df[main_df['station_id'] == station_id]
    #予測期間を指定
    test = test[test['date'] <= '2013-10-08 00:00:00']
    test = test[test['date'] >= '2013-09-30']
    #予測期間で不必要な説明変数の削除
    test = test.drop("station_id",axis=1)

    #今回LSTMモデルを使用するため、データを標準化
    #特徴量を標準化するための変数
    scaler = MinMaxScaler(feature_range=(0, 1))
    #標準化された出力をもとにスケールに変換(inverse)するために必要な変数
    scaler_for_inverse = MinMaxScaler(feature_range=(0, 1))
    #0から1の範囲に値を正規化して配列に直しているのが下の2行
    train_df_scale = scaler.fit_transform(train.iloc[:,2:])
    bikes_available_scale = scaler_for_inverse.fit_transform(train[["bikes_available"]])
    
    length = len(train_df_scale)
    train = train_df_scale[0:length,:]

    def create_dataset(dataset):
        dataX = []
        dataY = np.array([])

        #trainまたはtestの行を計算している。
        max_len = len(dataset)
        for i in range(24,max_len):
            xset = []
            #trainまたはtestの列を繰り返しに入れている。
            for j in range(1, dataset.shape[1]):
                a = dataset[i-24:i, j]
                xset.append(a)
            #各行の正規化したバイク台数を取ってくる
            temp_array = np.array(dataset[i:i+1,0])
            #各行からバイク台数だけを取ってきたものをまとめている
            dataY = np.concatenate([dataY,temp_array])
            dataX.append(xset)
            #dataYの形を変えてるだけ
        dataY = dataY.reshape(-1,1)
        return np.array(dataX), dataY 

    trainX, trainY = create_dataset(train)
    #LSTMのモデルに入力用にデータの形を整形
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))

    model = Sequential()
    model.add(LSTM(50, input_shape=(trainX.shape[1],24)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    hist = model.fit(trainX, trainY, epochs=300, batch_size=5, verbose=2)
    
    #学習済みモデルで予測
    train_predict = model.predict(trainX)
    #スケールをもとに戻す
    train_predict = scaler_for_inverse.inverse_transform(train_predict)

    #今回LSTMモデルを使用するため、データを標準化
    #特徴量を標準化するための変数
    scaler = MinMaxScaler(feature_range=(0, 1))
    #標準化された出力をもとにスケールに変換(inverse)するために必要な変数
    scaler_for_inverse = MinMaxScaler(feature_range=(0, 1))
    test_df_scale = scaler.fit_transform(test.iloc[:,2:])
    bikes_available_scale = scaler_for_inverse.fit_transform(test[["bikes_available"]])
    
    length = len(test_df_scale)
    test = test_df_scale[0:length,:]
    testX, testY = create_dataset(test)
    #LSTMのモデルに入力用にデータの形を整形
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))
    #学習済みモデルで予測
    test_predict = model.predict(testX)
    #スケールをもとに戻す
    test_predict = scaler_for_inverse.inverse_transform(test_predict)